In [1]:
#!pip3 install tensorflow_gpu==2.0


import tensorflow as tf
print(tf.__version__)

2.2.0


In [16]:
!pip3 install ktrain==0.18

     |████████████████████████████████| 25.2MB 130kB/s 
  Created wheel for ktrain: filename=ktrain-0.18.0-cp36-none-any.whl size=25252289 sha256=7e1c62d0db037db45478fb1c0d5405193888546e8cfd49554a8cf28fe72a01f3
  Stored in directory: /root/.cache/pip/wheels/80/de/95/d74bb02fa07a9c51b8d9177cc78c2499250ac12185295d19e8
Successfully built ktrain
  Found existing installation: ktrain 0.19.0
    Uninstalling ktrain-0.19.0:
      Successfully uninstalled ktrain-0.19.0


In [18]:
import pandas as pd
from ktrain import text
import numpy as np
from sklearn.model_selection import train_test_split

ModuleNotFoundError: ignored

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
data = pd.read_csv('/content/drive/My Drive/Máster KSchool/TFM/data/filtered_df_with_imgNames.csv',usecols=['RATING','DESCRIPTION'])
data = data.apply(lambda x: x.str.replace(',','.'))

In [10]:
data

,DESCRIPTION,RATING
0,When struggling rapper Coco (Azealia Banks) en...,5.3
1,A man taking revenge on the killers of his loy...,6.4
2,The story of a young Viking boy who accompanie...,5.5
3,A dark fairy tale about a gang of five childre...,7.0
4,Four frames of simultaneous action that altern...,6.0
...,...,...
107176,Add a Plot Ã‚Â»,7.3
107177,One of only two survivors from a Martian exped...,5.3
107178,Add a Plot Ã‚Â»,5.1
107179,Is having fun and saying what you really think...,5.5


In [11]:
data['RATING']  = data['RATING'].astype(float)
data['RATING'] = pd.cut( data['RATING'], bins = [0, 5, 6.5, 10], labels=['0','1','2'])
data

,DESCRIPTION,RATING
0,When struggling rapper Coco (Azealia Banks) en...,1
1,A man taking revenge on the killers of his loy...,1
2,The story of a young Viking boy who accompanie...,1
3,A dark fairy tale about a gang of five childre...,2
4,Four frames of simultaneous action that altern...,1
...,...,...
107176,Add a Plot Ã‚Â»,2
107177,One of only two survivors from a Martian exped...,1
107178,Add a Plot Ã‚Â»,1
107179,Is having fun and saying what you really think...,1


In [12]:
print(len(data[data['RATING']=='2']))
print(len(data[data['RATING']=='1']))
print(len(data[data['RATING']=='0']))

40270
47141
19770


In [13]:
MODEL_NAME = 'distilbert-base-uncased'
classes=['0','1','2']
t = text.Transformer(MODEL_NAME, maxlen=500, classes=classes)

NameError: ignored

In [ ]:
model = t.get_model()
model.summary()

In [ ]:
X = data.DESCRIPTION.values
Y = data.RATING.values

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
x_train, x_test, y_train, y_test= train_test_split(X, Y.astype(str), stratify=Y, test_size=0.2)

In [ ]:
x_train = list(x_train)
x_test = list(x_test)

In [ ]:
y_train = np.array(y_train).astype(int)
y_test = np.array(y_test).astype(int)

In [ ]:
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)

In [ ]:
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=128)

In [ ]:
### To freeze Distilbert weights
dBert = model.layers[0]
dBert_transformer1 = dBert.transformer.layer[0]
dBert.trainable = False

In [ ]:
learner.fit_onecycle(5e-5, 3)

In [ ]:
learner.validate(class_names=t.get_classes())